# Lab 3: Chatbot and Semantic Search Applications

---

본 모듈에서는 SageMaker 훈련 인스턴스에서 훈련한 모델을 로컬 공간으로 복사하여 추론을 수행합니다.

SageMaker 훈련된 모델 파라메터는 model.tar.gz로 압축되어 S3에 저장되며, 압축 파일 내에는 훈련 인스턴스의 /opt/ml/model의 모든 파일/디렉토리들이 포함되어 있습니다. 따라서, 로컬/개발/온프레미스 환경에서 훈련된 모델을 자유롭게 테스트할 수 있습니다.

만약 로컬/개발/온프레미스 환경이 아닌 SageMaker 엔드포인트 배포를 고려한다면, 아래 URL을 참조하세요.
- SageMaker Hugging Face Inference Toolkit: https://github.com/aws/sagemaker-huggingface-inference-toolkit
- Amazon SageMaker Deep Learning Inference Hands-on-Lab: https://github.com/aws-samples/sagemaker-inference-samples-kr


<div class="alert alert-info"><h4>Note</h4><p>
CUDA out of memory 에러가 발생하면, 현재 실행 중인 노트북들을 모두 셧다운 후, 이 노트북을 재실행해 주시기 바랍니다. 
</p></div>



In [ ]:
%load_ext autoreload
%autoreload 2
%store -r

In [ ]:
try:
    s3_model_path 
    local_model_dir 
    train_dir 
    print("[OK] You can proceed.")
except NameError:
    print("+"*60)
    print("[ERROR] Please run previous notebooks and before you continue.")
    print("+"*60)

In [ ]:
%%bash -s "$local_model_dir" "$s3_model_path"
#aws s3 cp $2 $1
cd $1
tar -xzvf model.tar.gz

In [ ]:
import numpy as np
import pandas as pd
import random
import time
from operator import itemgetter 

def get_faiss_index(emb, data, dim=768):
    import faiss
    n_gpus = torch.cuda.device_count()

    if n_gpus == 0:
        # Create the Inner Product Index
        index = faiss.IndexFlatIP(dim)
    else:
        flat_config = []
        res = [faiss.StandardGpuResources() for i in range(n_gpus)]
        for i in range(n_gpus):
            cfg = faiss.GpuIndexFlatConfig()
            cfg.useFloat16 = False
            cfg.device = i
            flat_config.append(cfg)

        index = faiss.GpuIndexFlatIP(res[0], dim, flat_config[0])

    index = faiss.IndexIDMap(index)
    index.add_with_ids(emb, np.array(range(0, len(data)))) 
    return index


def search(model, query, data, index, k=5, random_select=False, verbose=True):
    t = time.time()
    query_vector = model.encode(query)
    dists, top_k_inds = index.search(query_vector, k)
    if verbose:
        print('total time: {}'.format(time.time() - t))
    results = [itemgetter(*ind)(data) for ind in top_k_inds] 
    
    if random_select:
        return [random.choice(r) for r in results]
    else:
        return results

<br>

## 1. Load trained model
---

이전 모듈에서 훈련한 모델을 로드합니다. 

In [ ]:
import os
import torch
from sentence_transformers import SentenceTransformer

dirs = ([name for name in os.listdir(local_model_dir) if os.path.isdir(os.path.join(local_model_dir, name))])
model = SentenceTransformer(os.path.join(local_model_dir, dirs[0]))

<br>

## 2. Predictions
---

### Chatbot

챗봇은 크게 두 가지 형태로 개발합니다. 1) 생성 모델을 사용하여 해당 질문에 대한 창의적인 답변을 생성하거나, 2) 수많은 질문-답변 리스트들 중 질문에 부합하는 질문 후보들을 추린 다음 해당 후보에 적합한 답변을 찾는 방식이죠.
본 핸즈온은 2)의 방법으로 간단하게 챗봇 예시를 보여드립니다. 질문 텍스트를 입력으로 받으면, 해당 질문의 임베딩을 계산하여 질문 임베딩과 모든 질문 리스트의 임베딩을 비교하여 유사도가 가장 높은 질문 후보들을 찾고, 각 후보에 매칭되는 답변을 찾습니다.

코사인 유사도를 직접 계산할 수도 있지만, 페이스북에서 개발한 Faiss 라이브러리 (https://github.com/facebookresearch/faiss) 를 사용하면 훨씬 빠른 속도로 계산할 수 있습니다. Faiss는 Product 양자화 알고리즘을 GPU로 더욱 빠르게 구현한 라이브러리입니다.

References
- Billion-scale similarity search with GPUs: https://arxiv.org/pdf/1702.08734.pdf
- Product Quantizers for k-NN Tutorial Part 1: https://mccormickml.com/2017/10/13/product-quantizer-tutorial-part-1
- Product Quantizers for k-NN Tutorial Part 1: http://mccormickml.com/2017/10/22/product-quantizer-tutorial-part-2

In [ ]:
chatbot_df = pd.read_csv(f'{train_dir}/chatbot-train.csv')
chatbot_data = chatbot_df['A'].tolist()

#### Load embedding & Indexing the dataset

In [ ]:
chatbot_emb = np.load(f'{local_model_path}/chatbot_emb.npy')
chatbot_index = get_faiss_index(chatbot_emb, chatbot_data)

#### Inference
샘플 질문들에 대한 추론을 수행합니다. 챗봇 답변은 계속 변경됩니다.

In [ ]:
query = ['커피 라떼 마시고 싶어']
search(model, query, chatbot_data, chatbot_index, k=10, random_select=True)

In [ ]:
query = ['너무 졸려', '놀고 싶어']
search(model, query, chatbot_data, chatbot_index, random_select=True)

### Semantic Search (News)

In [ ]:
news_data = []
f = open(f'{train_dir}/KCCq28_Korean_sentences_EUCKR_v2.txt', 'rt', encoding='cp949')
lines = f.readlines()
for line in lines:
    line = line.strip()
    news_data.append(line)
f.close()

#### Load embedding & Indexing the dataset

In [ ]:
news_emb = np.load(f'{local_model_path}/news_emb.npy')
news_index = get_faiss_index(news_emb, news_data)

#### Inference
샘플 질문들에 대한 추론을 수행합니다. 

In [ ]:
query = ['메이저리그 베이스볼']
search(model, query, news_data, news_index, k=10, random_select=False)

In [ ]:
query = ['아이스 라떼', '미세먼지']
search(model, query, news_data, news_index, k=7, random_select=False)